<a href="https://colab.research.google.com/github/JozefSL/pyNotes/blob/main/eia/weeklyCrudeStocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd
import requests
import plotly.express as px
import os

In [12]:
from google.colab import userdata
api_key = userdata.get('api_key')

In [14]:
url = 'https://api.eia.gov/v2/petroleum/stoc/wstk/data/?api_key='+ api_key + '&data[0]=value&frequency=weekly&start=2014-01-01&sort[0][column]=period&sort[0][direction]=asc&facets[series][]='

In [15]:
response = requests.get(url + 'WCESTUS1')

In [16]:
df = pd.DataFrame(response.json().get('response').get('data'))[['period','value']]
df.columns = ['Period', 'value']
df['Period'] = df['Period'].apply(pd.to_datetime)
df.value = (df.value.astype('Int64')/1000).astype('Int64')
df['year'] = df['Period'].dt.year
df['week'] = df['Period'].dt.isocalendar().week
df.tail()

,Period,value,year,week
556,2024-08-30,418,2024,35
557,2024-09-06,419,2024,36
558,2024-09-13,417,2024,37
559,2024-09-20,413,2024,38
560,2024-09-27,416,2024,39


In [17]:
dfA = df.pivot(index='week', columns='year', values='value').astype('Int64') #.reset_index(drop='true')
dfA = dfA.drop(dfA.index[-1]).reset_index()  #eliminate the add years with 53 weeks

In [18]:
dfAA = pd.DataFrame(dfA.iloc[:, -6:-1].mean(axis=1))
dfAA.columns = ['5yAvg']
dfAA['5yMin'] = dfA.iloc[:, -6:-1].min(axis=1)
dfAA['5yMaxA'] = dfA.iloc[:, -6:-1].max(axis=1)
dfAA['5yMax'] = dfAA['5yMaxA']-dfAA['5yMin']
dfAA['10yAvg'] = dfA.iloc[:, -11:-1].mean(axis=1)
dfAA['10yMin'] = dfA.iloc[:, -11:-1].min(axis=1)
dfAA['10yMax'] = dfA.iloc[:, -11:-1].max(axis=1)
dfAA['week'] = dfA['week']

In [19]:
dfA['weekDate'] = pd.to_datetime(str(pd.Timestamp.now().year)+dfA['week'].astype(str)+'5',format='%Y%W%w')
dfAA['weekDate'] = pd.to_datetime(str(pd.Timestamp.now().year)+dfAA['week'].astype(str)+'5',format='%Y%W%w')

In [20]:
# Create the area chart
fig = px.area(dfAA, x=dfAA.weekDate, y=['5yMin', '5yMax'], range_y=[400, 550], #range_x=[dfAA.index.min()-.2, dfAA.index.max()+.2],
              title='Stacked Area Chart with Line Chart').update_traces(fillcolor='rgba(128, 128, 128, 0.2)')

fig.update_traces(line_color='rgba(0,0,0,0)',showlegend=False)

fig.add_scatter(x=dfAA.weekDate, y=dfAA['5yAvg'], mode='lines', name='5yAvg', line=dict(color='black', width=2))
fig.add_scatter(x=dfA.weekDate, y=dfA[2023], mode='lines', name='2023',line=dict(color='blue', width=2))
fig.add_scatter(x=dfA.weekDate, y=dfA[2024], mode='lines+markers', name='2024',line=dict(color='red', width=3), marker=dict(color='red'))

# Customize the chart
fig.update_layout(
    autosize=False,
    width=1200,
    height=800,
    title={
        'text': '<b>Total Crude Oil Stocks Excluding SPR</b>',
        'font': {'size': 28, 'family': 'Arial', 'color': 'black'}
        },
    xaxis_title='Week',
    yaxis_title='Crude Oil Stocks (MMbbl)'

)


fig.update_traces(fillcolor='rgba(0,0,0,0)', selector=dict(name='5yMin'))


fig.add_scatter(x = [dfA['weekDate'][dfA[2024].isna().idxmax()-1]], y = [dfA[2024][dfA[2024].isna().idxmax()-1]],
                     mode = 'markers + text',
                     marker = {'color':'red', 'size':14},
                     showlegend = False,
                     text = [dfA[2024][dfA[2024].isna().idxmax()-1]],
                     textfont=dict(color='red', size=16),
                     textposition='bottom right') # top, bottom, middle


fig.show()